# Introduction
- Our team's project is to create coding to do data analysis.
- Our team members are 김동구, 김민재, 박찬근.
- The role of team members is to develop code and communicate with customer. And professor's role is a mentor and client for our team.



# Objective of project
- This objective of project is to analyze the received raw data and deliver it in the form of a documentation file.


# Structure of code
![flow_chart.png](./image/flow_chart.png)
- 위 그림은 이 프로그램의 전체적인 flow chart입니다. src module 7개와 run.py 이렇게 총 8개 module로 구성되어 있습니다.
- The figure above is the overall flow chart of this program. It consists of a total of 8 modules : 7 src modules and run.py.
    
- 프로그램은 run.py를 실행하게 되면 process.py에서 여러 module들이 연산하고 plot한 data들을 하나로 통합하여 저장합니다.
- When the program runs run.py, In process.py, several modules calculate. And plotted data are saved into integrated form. 

- 이제부터 각 module에 대해 설명하겠습니다.
- From now, we will explain each module.

##  1. run.py
  - 프로그램을 시동하는 역할을 하는 module입니다. data folder에 분석할 data를 넣고 run.py를 실행하게 되면 프로그램의 중심이 되는 process.py를 import해 프로그램을 실행하게 됩니다.
  - run.py is the module responsible for running the program. When you put the data to be analyzed in the data folder and run run.py, the program is operated by importing process.py.

##  2. process.py
  - 프로그램의 중심이 되는 module입니다. data folder에 있는 raw data xml 파일들을 파싱하고 plot module과 fitting module에 data를 전달하고 이 module들이 plot한 figure를 모아 png 파일로 저장하고 이외의 정보들을 csv 파일로 저장하는 역할을 합니다.
  - process.py is the central module of the program. It parses raw data xml files in the data folder and delivers the data to the plot module and fitting module, collects the plotted figure, and saves it as a png file and saves other information as a csv file.
![process.png](./image/process.png)
    - raw data를 모듈에 주고 plot된 figure를 받아오는 코드
    - Code that gives raw data to the module and receives the plotted figure.

## 3. data_set.py
![data_set.png](./image/data_set.png)
  - 각 raw data들의 str 형식으로 나열되어 있는 값들을 연산이 가능한 float으로 구성된 리스트로 바꿔주는 module.
  -Module that converts the values listed in str format of each raw data into a list of float that can be calculated on.
  
## 4. R_square.py
![R_square.png](./image/R_square.png)
  - fitting 함수의 Rsq값을 계산해주는 module.
  - Module that calculates Rsq value of the fitting function.

## 5. poly_fitting.py
![poly_fitting.png](./image/poly_fitting.png)
  - numpy의 polyfit을 이용하여 입력받은 raw data를 다항함수로 fitting 해주는 module. for문의 시작 숫자부터 number까지의 차수로 피팅하고 각각을 리스트에 저장한다.
  - Module that fitting the inputed raw data with a polynomial function using numpy's polyfit. Fitting from the starting number of for loop to the order of number and store each in a list.

## 6. ILplot.py
![ILplot1.png](./image/ILplot1.png)
![ILplot2.png](./image/ILplot2.png)
  - Intensity와 Wavelength의 raw data plot, reference fitting function plot, fitting 함수로 raw data를 가공한 plot까지 총 3개의 그래프를 그리는 module. 정의된 함수 이름과 역할이 같다.
  - Module that draws a total of 3 graphs, including Intestiy and Wavelength raw data plot, reference fitting function plot, and plot processed with raw data by fitting function. The defined function name and role are same.

## 7. exp_fitting.py
![exp_fitting.png](./image/exp_fitting.png)
  - IV fitting의 경우 shockley equation 만으로 fitting하기 힘들기 때문에 다항함수 항을 추가하여 fitting한다. 앞서 설명한 poly_fitting.py에서 다항함수 피팅을 먼저 한 후 하나씩 exp 함수와 더해 각각 fitting하고 Rsq를 비교해 가장 정확한 fitting함수를 찾고 plot까지 진행하는 module.
  - In the case of IV fitting, It is difficult to fitting only with the shockley equation, so that polynomial function term is added for fitting. 
  - Module in poly_fittinf.py, polynomial function fitting is first perfomed, then each is added with the exp function to fitting each and compare Rsq to find the most accurate fitting function and proceed to the plot. 
    
  
## 8. IVplot.py
![IVplot.png](./image/IVplot.png)
  - IV의 raw data를 plot하는 module.
  - Module that plot raw data of IV

# Example of analysis result
  - sample image of analysis result
![example_figure.png](./image/example_figure.png)

  - a table in csv file
![example_csv.png](./image/example_csv.png)

# Result of wafer-to-wafer using csv file